### Given a youtube URL the following will download transcript then summarize it

In [ ]:
from __future__ import unicode_literals
import openai
import os
from dotenv import load_dotenv
import youtube_dl
from youtube_transcript_api import YouTubeTranscriptApi
import shutil
from pytube import YouTube
from pydub import AudioSegment
import time
import re
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

# video_url = 'https://www.youtube.com/watch?v=SNgoul4vyDM'
# summary_title = "ufo_hearing_whisper_download_transcript"

video_url = 'https://www.youtube.com/watch?v=34wA_bdG6QQ'
summary_title = "lex_fridman_391"

video_id = video_url.split('=')[1]
task = "Please provide a detailed summary of the following: \n"

chunk_size = 2300
input_tokens = 0
output_tokens = 0
models = ['gpt-3.5-turbo', 'gpt-4']
USE = 0
TRANSCRIBE = False

In [ ]:
def download_audio_from_youtube(video_url, output_path, file_name="audio.mp4"):
    try:
        yt = YouTube(video_url)
        audio_stream = yt.streams.filter(only_audio=True).first()
        audio_stream.download(output_path, filename=file_name)
        print("Audio download completed!")
    except Exception as e:
        print(f"An error occurred: {e}")


def split_audio_into_chunks(input_file, output_folder, chunk_duration=900):
    audio = AudioSegment.from_file(input_file)
    audio_duration_ms = len(audio)
    for start_time_ms in range(0, audio_duration_ms, chunk_duration * 1000):
        end_time_ms = start_time_ms + (chunk_duration * 1000)
        chunk = audio[start_time_ms:end_time_ms]
        location = f"{output_folder}/audio_chunks"
        if not os.path.exists(location):
            os.makedirs(location)
        output_file = f"{output_folder}/audio_chunks/chunk_{start_time_ms//1000}.mp4"
        chunk.export(output_file, format="mp4")


def get_video_title(url):
    with youtube_dl.YoutubeDL({}) as ydl:
        info_dict = ydl.extract_info(url, download=False)
        return info_dict.get('title', None)


def summarize(input, model=models[USE]):
    global input_tokens, output_tokens
    if model == "gpt-3.5-turbo":
        input_cost = 0.0015/1000
        output_cost = 0.002/1000
    else:
        input_cost = 0.03/1000
        output_cost = 0.06/1000

    completion = openai.ChatCompletion.create(
        model=model,
        messages=[
          {"role": "system", "content": "I am going to be providing you with chunks of text and I want you to summarize it for me. Sometimes I will also be giving you a summary of a previous chunk of text. Please only summarize the new chunk of text while also taking into account the previous summary. Also, don't ever mention the text or chunk of text or the summary in your response. Keep as much detail as possible."},
          {"role": "user", "content": input}])
    input_tokens += completion.usage.prompt_tokens
    output_tokens += completion.usage.completion_tokens

    cost = input_tokens*input_cost + output_tokens*output_cost
    print(f"Tokens thus far: {input_tokens + output_tokens} with a cost of ${round(cost, 4)}")
    reply_content = completion.choices[0].message.content
    return reply_content


def create_directory(dir_name):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

def download_transcript(video_id):
    video_id = video_id.split('=')[-1]
    transcript = YouTubeTranscriptApi.get_transcript(video_id)

    with open(f"{summary_title}/transcript.txt", "w") as file:
        for line in transcript:
            file.write(line['text'] + '\n')

def chunk_transcript_sentences(file_name, chunk_size, summary_title):
    # Open and read the file
    with open(file_name, 'r') as file:
        text = file.read().replace('\n', ' ')
    
    # Split the text into sentences
    sentences = re.split('(?<=[.!?]) +', text)

    chunks, chunk = [], []
    current_chunk_size = 0

    for sentence in sentences:
        sentence_words = sentence.split(' ')
        sentence_length = len(sentence_words)
        
        # If adding the next sentence doesn't exceed the chunk size, add it to the current chunk
        if current_chunk_size + sentence_length <= chunk_size:
            chunk.append(sentence)
            current_chunk_size += sentence_length
        else:
            # Otherwise, finish the current chunk and start a new one
            chunks.append(' '.join(chunk))
            chunk = [sentence]
            current_chunk_size = sentence_length

    # Add the last chunk if it's non-empty
    if chunk:
        chunks.append(' '.join(chunk))

    create_directory(f"{summary_title}/chunks")

    # Write each chunk to a file
    for i, chunk in enumerate(chunks):
        with open(f'{summary_title}/chunks/chunk{i}.txt', 'w') as file:
            file.write(chunk)

def chunk_transcript(file_name):
    with open(file_name, 'r') as file:
        text = file.read().replace('\n', ' ')
    words = text.split(' ')
    chunks = [words[i:i + chunk_size]
              for i in range(0, len(words), chunk_size)]
    create_directory(f"{summary_title}/chunks")
    for i, chunk in enumerate(chunks):
        with open(f'{summary_title}/chunks/chunk{i}.txt', 'w') as file:
            file.write(' '.join(chunk))


def get_sorted_files_by_date(directory):
    file_list = os.listdir(directory)
    files_with_mtime = [(file, os.path.getmtime(
        os.path.join(directory, file))) for file in file_list]
    sorted_files = sorted(files_with_mtime, key=lambda x: x[1])
    sorted_filenames = [file[0] for file in sorted_files]
    return sorted_filenames


def read_text_from_files(directory_path):
    text_array = []
    files = get_sorted_files_by_date(directory_path)
    for file_name in files:
        file_path = os.path.join(directory_path, file_name)
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                text_array.append(file.read())
    return text_array


def read_and_join_text_from_files(directory_path):
    joined_text = ""
    files = get_sorted_files_by_date(directory_path)
    for file_name in files:
        file_path = os.path.join(directory_path, file_name)
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                joined_text += file.read() + "\n"
    return joined_text

In [ ]:
if os.path.exists(summary_title):
  shutil.rmtree(summary_title)
create_directory(summary_title)

### Download audio then transcribe using openai whisper api

In [ ]:
if TRANSCRIBE:
    download_audio_from_youtube(video_url, summary_title)
    split_audio_into_chunks(summary_title+"/audio.mp4",
                            summary_title, chunk_duration=1200)

In [ ]:
if TRANSCRIBE:
    files = get_sorted_files_by_date(summary_title+"/audio_chunks")
    print(f"Chunks to transcribe: {files}")
    if os.path.exists(f"{summary_title}/transcript.txt"):
        os.remove(f"{summary_title}/transcript.txt")
    for file in files:
        print(f"Transcribing {file}")
        file = open(summary_title + "/audio_chunks/" + file, "rb")
        transcript = "\n" + openai.Audio.transcribe("whisper-1", file).text
        with open(f"{summary_title}/transcript.txt", "a") as f:
            f.write(transcript)
else:
    download_transcript(video_id)

### Summarize each chunk

In [ ]:
if os.path.exists(summary_title+"/chunks"):
    shutil.rmtree(summary_title+"/chunks")
create_directory(summary_title+"/chunks")
# chunk_transcript(f'{summary_title}/transcript.txt', chunk_size, summary_title)
chunk_transcript(f'{summary_title}/transcript.txt')
data = read_text_from_files(f"{summary_title}/chunks")

In [ ]:
if os.path.exists(summary_title+"/summaries"):
    shutil.rmtree(summary_title+"/summaries")
create_directory(f"{summary_title}/summaries")

if os.path.exists(summary_title+"/instructions"):
    shutil.rmtree(summary_title+"/instructions")
create_directory(f"{summary_title}/instructions")

print(f"Summarizing {len(data)} articles")
multiple_summaries = len(data) > 1
for i in range(len(data)):
    previous_summary = ""
    if i > 0:
        with open(f"{summary_title}/summaries/summary{i-1}.txt", "r") as f:
            previous_summary = "Please provide a detailed (keep as much detail as possible) summary while keeping the following context in mind: \n" + f.read().replace('\n', ' ') + "\n\n"
    else:
        previous_summary = "Please provide a detailed summary of the following while keeping as much details in as possible: \n"
    instruction = previous_summary + data[i]
    with open(f"{summary_title}/instructions/instruction_{i}.txt", "w") as file:
        file.write(instruction)
    print(f"Summarizing article {i} with {len(instruction.split(' '))} words")
    response = summarize(instruction)
    with open(f"{summary_title}/summaries/summary{i}.txt", "w") as f:
        f.write(response)

### Join summaries together then summarize them altogether using GPT-4

In [ ]:
directory_path = f'{summary_title}/summaries'
joined_text_variable = read_and_join_text_from_files(directory_path)
files = get_sorted_files_by_date(directory_path)
multiple_summaries = len(files) > 1
with open(f'{summary_title}/summaries/joined_text.txt', 'w') as file:
    file.write(joined_text_variable)
if multiple_summaries:
    print(f"Summarizing joined summaries with {len(joined_text_variable.split(' '))} words")

    instruction = "Make the following summaries flow as one long and detailed article: \n" + joined_text_variable.rstrip()
    with open(f"{summary_title}/instructions/instruction.txt", "w") as f:
        f.write(instruction)

    completion = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": instruction}])
    sum = completion.choices[0].message.content
else:
    sum = joined_text_variable
with open(f"{summary_title}/summary.txt", "w") as f:
    f.write(sum)

In [ ]:
import boto3
voice_id = "Matthew"
output_format = "mp3"
import pygame

if os.path.exists(summary_title+"/audio"):
    shutil.rmtree(summary_title+"/audio")
create_directory(f"{summary_title}/audio")

# Create an Amazon Polly client
polly_client = boto3.Session(
    aws_access_key_id=os.getenv("polly_access_key_id"),
    aws_secret_access_key=os.getenv("polly_secret_key"),
    region_name='ca-central-1').client('polly')

def generate_audio(input, count):
  start_time = time.time()
  audio = polly_client.synthesize_speech(
      Text=input,
      VoiceId=voice_id,
      OutputFormat=output_format,
      Engine="neural"
      )
  save_audio(audio, count)
  return audio

def save_audio(response, count):
  with open(f"{summary_title}/audio/story_{count}.mp3", "wb") as f:
      f.write(response["AudioStream"].read())

def read_audio(file):
  pygame.mixer.init()
  pygame.mixer.music.load(file)
  pygame.mixer.music.play()

  while pygame.mixer.music.get_busy():
    # Optional: add a delay to reduce CPU usage
    time.sleep(0.1)  

  return pygame.mixer.music

In [ ]:
with open(f"{summary_title}/summary.txt", "r") as f:
  story = f.read()
count = 0
for section in story.split('\n\n'):
  audio = generate_audio(section, count)
  count += 1

In [ ]:
for i in range(count):
  section = story.split('\n\n')[i]
  print(i, section)
  media_player = read_audio(f"{summary_title}/audio/story_{i}.mp3")